In [3]:
import os
import sys
import requests
import os, base64
import time
import platform
import json

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
import pandas as pd
import io

In [5]:
host = "https://0.0.0.0:9600"

headers = {
    'Content-Type': 'application/json',
    'accept': 'application/json',
}

cert=('/opt/vault/server-sberca-cert.pem',
      '/opt/vault/server-sberca-key.pem')

r = requests.get(host + "/liveness", headers={"Content-Type": "application/json"}, cert=cert, verify=False)
response = r.json()

In [6]:
response

[{'health_status': 'running'}, 200]

In [7]:
with open("./meta/doc_data.jsonl", 'r', encoding='utf-8') as fd:
    data = [json.loads(line) for line in fd]

In [8]:
data[0]

{'doc_name': 'doc_5.pdf',
 'page': '0',
 'text': 'Государственная  \nавтоматизированная  \nинформационная  \nсистема  \n«ЭРА-ГЛОНАСС» \n'}

In [9]:
files = {}
for doc in data:
    if doc["doc_name"] in files.keys():
        files[doc["doc_name"]] += "\n" + doc["text"]
    else:
        files[doc["doc_name"]] = doc["text"]

In [10]:
files.keys()

dict_keys(['doc_5.pdf', 'doc_4_ydx.pdf', 'sber-ar-78-88-2023-ru.pdf', 'x5_sr2022_rus_30_40.pdf', 'sber-ar-20-30-2023-ru.pdf', 'RUS_NOVATEK_AR22_29_39.pdf', 'x5_sr2022_rus_46_56.pdf', 'doc_10.pdf', 'ant_workers.pdf', 'sber-ar-142-152-2023-ru.pdf', 'sber-ar-6-16-2023-ru.pdf', 'RUS_NOVATEK_AR22_9_19.pdf', 'sber-ar-131-141-2023-ru-1.pdf', 'sber-ar-90-100-2023-ru.pdf', 'doc_9.pdf', 'doc_news_ugra.pdf', 'doc_8.pdf', 'sber-ar-42-52-2023-ru.pdf'])

In [11]:
one_file = ""
for text in files.values():
    one_file += "\n" + text

In [12]:
len(one_file)

446170

In [23]:
with open("one_large.txt", "a", encoding="utf-8") as file:
    file.write(one_file)

In [24]:
with open("./meta/one_large.pdf", 'rb') as f:
    qa_data = f.read()

In [25]:
response = requests.post(f'{host}/create_task/chat', headers=headers, data=qa_data, cert=cert, verify=False)

print(f"Status: {response.status_code}. Result: {response.text}")

json.loads(response.text)["taskId"]

Status: 200. Result: {"taskId":"912467c2-2cb4-472e-b738-641287ebd793","status":"wait","error":""}


'912467c2-2cb4-472e-b738-641287ebd793'

In [26]:
taskId = json.loads(response.text)["taskId"]

In [29]:
response = requests.post(f'{host}/get_status/chat', headers=headers, json={"taskId": taskId}, cert=cert, verify=False)

In [30]:
response.text

'{"status":"done","error":"","Id":"912467c2-2cb4-472e-b738-641287ebd793"}'

In [133]:

json_data = {
    "taskId": taskId,
    "history": [],
    "question": "Ущерб государству от травмы одного человека ?",
    "model": "GigaChat-Pro"
}

response = requests.post(f'{host}/create_task/question', headers=headers, json=json_data, cert=cert, verify=False)

In [134]:
replyId = json.loads(response.text)["replyId"]

In [135]:
json_data = {
    'replyId': replyId,
}

response = requests.post(f'{host}/get_status/question',  headers=headers, json=json_data, cert=cert, verify=False)
json.loads(response.text)["status"]

{'status': 'done', 'error': '', 'Id': '65037734-83e8-42f4-bddb-f3835ac967f3'}

In [137]:

json_data = {
    'replyId': replyId,
}

response = requests.post(f'{host}/get_result/question',  headers=headers, json=json_data, cert=cert, verify=False)
json.loads(response.text)['result']['answer']

In [138]:
json.loads(response.text)['result']['answer']

'В контексте не содержится информации, позволяющей уточнить или исправить ответ относительно ущерба государству от травмы одного человека. Поэтому ответ остается прежним: **ущерб государству от травмы одного человека оценивается в среднем в 8,2 млн рублей**.'

In [139]:
json_data = {
    "taskId": taskId,
}
response = requests.post(f'{host}/clear_result/chat', headers=headers, json=json_data, cert=cert, verify=False)

print(f"Status: {response.status_code}. Result: {response.text}")


Status: 200. Result: null


In [31]:
questions_df = pd.read_excel('./meta/questions.xlsx')
questions = questions_df.to_dict()

In [32]:
questions["doc_name"]

{0: 'sber-ar-6-16-2023-ru.pdf',
 1: 'sber-ar-6-16-2023-ru.pdf',
 2: 'sber-ar-6-16-2023-ru.pdf',
 3: 'sber-ar-6-16-2023-ru.pdf',
 4: 'sber-ar-20-30-2023-ru.pdf',
 5: 'sber-ar-20-30-2023-ru.pdf',
 6: 'sber-ar-42-52-2023-ru.pdf',
 7: 'sber-ar-42-52-2023-ru.pdf',
 8: 'sber-ar-142-152-2023-ru.pdf',
 9: 'sber-ar-90-100-2023-ru.pdf',
 10: 'sber-ar-90-100-2023-ru.pdf',
 11: 'sber-ar-78-88-2023-ru.pdf',
 12: 'sber-ar-131-141-2023-ru-1.pdf',
 13: 'sber-ar-131-141-2023-ru-1.pdf',
 14: 'doc_10.pdf',
 15: 'doc_4_ydx.pdf',
 16: 'doc_5.pdf',
 17: 'x5_sr2022_rus_30_40.pdf',
 18: 'doc_9.pdf',
 19: 'x5_sr2022_rus_46_56.pdf',
 20: 'RUS_NOVATEK_AR22_9_19.pdf',
 21: 'RUS_NOVATEK_AR22_29_39.pdf',
 22: 'doc_4_ydx.pdf',
 23: 'doc_4_ydx.pdf',
 24: 'doc_news_ugra.pdf',
 25: 'doc_news_ugra.pdf',
 26: 'doc_news_ugra.pdf',
 27: 'ant_workers.pdf',
 28: 'ant_workers.pdf',
 29: 'doc_9.pdf',
 30: 'doc_9.pdf'}

In [33]:
# selected_questions_idx = [14,15,16,17,18,22,23,27,28,30]
# selected_questions = [list(questions['question'].values())[idx] for idx in selected_questions_idx]

In [36]:
# selected_questions
list(questions['question'].values())

['Сколько активных корпоративных клиентов ?',
 'Найди коэффицент CET1 по итогам 23 года',
 'Какую награду присудило агентство нильсон репорт ?',
 'Насколько вырос показатель CSI в процентах?',
 'Какова доля Сбербанка в сегменте средств частных клиентов ?',
 'Найди основные показатели по потребительскому кредитованию',
 'Как изменилась доля Сбера в брокерском обслуживании с 2022 по 2023 год',
 'Назови нововведения по теме финансового планирования в сегменте B2C',
 'Какой прогноз НКР присвоило СберБанку в 2023 году ?',
 'Найди чистый доход от комиссий за 2023 год ',
 'Найди показатели рентабельности, которые показали отрицательную динамику',
 'Для чего нужен gigacode ?',
 'Назови функции комитета по кадрам и вознаграждениям',
 'Назови задачи комитета по кадрам и вознаграждениям',
 'Назови цели развития магазинов у дома у ритейлера Лента',
 'Сколько всего было продано Станций Ядекса ?',
 'Ущерб государству от травмы одного человека ?',
 'Как x5 боролся с ковидом?',
 'Какие цели развития у

In [35]:
import time

In [37]:
for question in list(questions['question'].values()):
    json_data = {
        "taskId": taskId,
        "history": [],
        "question": question,
        "model": "GigaChat-Pro"
    }
    response = requests.post(f'{host}/create_task/question', headers=headers, json=json_data, cert=cert, verify=False)

    replyId = json.loads(response.text)["replyId"]
    print(replyId)

    status = 'wait'

    json_data = {
        'replyId': replyId,
    }
    while status != "done":
        time.sleep(4)
        response = requests.post(f'{host}/get_status/question',  headers=headers, json=json_data, cert=cert, verify=False)
        status = (json.loads(response.text)["status"])
        print(status)

        if status == "failed":
            break

    response = requests.post(f'{host}/get_result/question',  headers=headers, json=json_data, cert=cert, verify=False)
    print(json.loads(response.text)['result']['answer'])

73f3c8f1-3504-4ce5-8520-8ac1f0af716b
done
По состоянию на 2023 год у Сбера насчитывалось **3,2 миллиона активных корпоративных клиентов**.
bf8e4fef-5c34-4135-99c1-32101319f8a2
done
В представленном контексте отсутствует информация, позволяющая определить коэффициент CET1 по итогам 2023 года. Для получения точных данных рекомендуется обратиться к финансовой отчетности соответствующего банка за указанный период.
bfdd4395-deda-464f-9ca3-0e53768c0ce5
done
Из представленного контекста нельзя точно определить, какую награду присудило агентство Nielsen Report. В тексте упоминаются различные награды, но конкретно про награду от Nielsen Report ничего не сказано. Если у вас есть дополнительная информация или уточнения, пожалуйста, поделитесь ими!
9babbe11-d551-4a3b-8479-5d26f5fb8b8b
done
Контекст содержит информацию о том, что средняя оценка деятельности Наблюдательного совета за 2023 год составила 4,85 балла из 5 возможных, тогда как в предыдущем году она составляла 4,73 балла. Таким образом, р